In [1]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from os import walk
import pandas as pd
import fastparquet
import pyarrow.parquet as parquet
import numpy as np
# import seaborn as sns


In [ ]:
# folders= ['features/gt', 'features/test', 'features/sumb']
# pds = [None, None, None]

# num = 2
# for i in range(num):
#     pds[i] = parquet.read_table(folders[i], columns=['objectId']).to_pandas()
# ids=set(pd.concat(pds).objectId.values)

In [ ]:
# len(ids)
len(docs)

In [ ]:
# docs=[]
# for (dirpath, dirnames, filenames) in walk('../dataset/texts/train/'):
#     for name in filenames:
#         if name.startswith('part'):
#             texts = pd.read_parquet('../dataset/texts/train/' + name,columns = ['objectId','lang','preprocessed'], engine='fastparquet')
#             texts= texts[(texts.lang=='ru')]
#             for i, line in texts[['objectId','preprocessed']].values:
#                 docs.append(TaggedDocument([ i if type(i)==str else i.decode("utf-8") for i in line],[str(i)]))

test_texts = pd.read_parquet('../dataset/texts/test/',columns = ['objectId','lang','preprocessed'], engine='fastparquet')
test_texts= test_texts[(test_texts.lang=='ru')]
for i, line in test_texts[['objectId','preprocessed']].values:
    docs.append(TaggedDocument([ i if type(i)==str else i.decode("utf-8") for i in line],[str(i)]))



In [ ]:
import gc
del test_texts
gc.collect()

In [ ]:
len(docs)

In [ ]:
model = Doc2Vec( vector_size = 20 # Model initialization
    , dm=0      
    , window = 20
    , min_count = 150
    , workers = 24)

model.build_vocab(docs) # Building vocabulary


In [ ]:
len(model.wv.vocab)

In [ ]:
alpha_val = 0.065       # Initial learning rate
min_alpha_val = 1e-4     # Minimum for linear learning rate decay
passes = 4              # Number of passes of one document during training

alpha_delta = (alpha_val - min_alpha_val) / (passes - 1)

In [ ]:
import gc
gc.collect()

In [ ]:
import random
for epoch in range(passes):

    # Shuffling gets better results

    random.shuffle(docs)

    # Train

    model.alpha, model.min_alpha = alpha_val, alpha_val



    print('Start %i at alpha %f' % (epoch + 1, alpha_val))
    
    model.train(docs, epochs=3, total_examples=int(model.corpus_count))
            
    x1 = set([int(a[0]) for a in model.docvecs.most_similar(positive=[model.infer_vector(['кемеров','вишн','трагед','погибш','пожар','зимн'])], topn=5)])
    for d in docs:
        if int(d.tags[0]) in x1 :
            print(' '.join(d.words[:10]))
    
    print('-------------')
    x1 = set([int(a[0]) for a in model.docvecs.most_similar(positive=[model.infer_vector(['иисус','икон','бог','бож','свят'])], topn=5)])
    for d in docs:
        if int(d.tags[0]) in x1 :
            print(' '.join(d.words[:10]))
            
    print('-------------')     
    x1 = set([int(a[0]) for a in model.docvecs.most_similar(positive=[model.infer_vector(['приготовлен','сливочн','ингредиент','чеснок','репчат'])], topn=5)])
    for d in docs:
        if int(d.tags[0]) in x1 :
            print(' '.join(d.words[:10]))

    
    
#     x1 = set([int(a[0]) for a in model.docvecs.most_similar(positive=[model.infer_vector(["сыночк", "маленьк", "ласков"])], topn=5)])
#     print([' '.join(_) for _ in all_[(all_.objectId.map(lambda x : x in x1))].preprocessed.values])
#     x1 = set([int(a[0]) for a in model.docvecs.most_similar(positive=[model.infer_vector(["ударн", "тренировк", "красот"])], topn=5)])
#     print([' '.join(_) for _ in all_[(all_.objectId.map(lambda x : x in x1))].preprocessed.values])
#     x1 = set([int(a[0]) for a in model.docvecs.most_similar(positive=[model.infer_vector(['киев', 'торгов', 'развлекательн', 'центр', 'нарушен', 'пожарн', 'безопасн', 'будут', 'закрыва', 'суд', 'киевск', 'горадминистрац', 'намер', 'добива', 'суд', 'прекращен', 'работ', 'торгов', 'центр', 'котор', 'выявл', 'нарушен', 'норм', 'пожарн', 'безопасн', 'киевск', 'горадминистрац', 'намер', 'добива', 'суд', 'прекращен', 'работ', 'торгов', 'центр', 'котор', 'выявл', 'нарушен', 'норм', 'пожарн', 'безопасн', 'киев', 'торгов', 'развлекательн', 'центр', 'нарушен', 'пожарн', 'безопасн', 'будут', 'закрыва', 'суд'])], topn=5)])
#     print([' '.join(_) for _ in all_[(all_.objectId.map(lambda x : x in x1))].preprocessed.values])

    print('Completed pass %i at alpha %f' % (epoch + 1, alpha_val))

    # Next run alpha

    alpha_val -= alpha_delta

In [ ]:
    x1 = set([int(a[0]) for a in model.docvecs.most_similar(positive=[model.infer_vector(['кемеров','вишн','трагед','погибш','пожар','зимн'])], topn=5)])
    for d in docs:
        if int(d.tags[0]) in x1 :
            print(' '.join(d.words[:10]))
    
    print('-------------')
    x1 = set([int(a[0]) for a in model.docvecs.most_similar(positive=[model.infer_vector(['иисус','икон','бог','бож','свят'])], topn=5)])
    for d in docs:
        if int(d.tags[0]) in x1 :
            print(' '.join(d.words[:10]))
            
    print('-------------')     
    x1 = set([int(a[0]) for a in model.docvecs.most_similar(positive=[model.infer_vector(['приготовлен','сливочн','ингредиент','чеснок','репчат'])], topn=5)])
    for d in docs:
        if int(d.tags[0]) in x1 :
            print(' '.join(d.words[:10]))

In [ ]:

with open('doc2vec.csv', 'a') as f:
    for (dirpath, dirnames, filenames) in walk('../dataset/texts/train/'):
        for name in filenames:
            if name.startswith('part'):
                # Read single part
                texts = pd.read_parquet('../dataset/texts/train/' + name,columns = ['objectId','preprocessed'], engine='fastparquet')        
                texts['preprocessed']=texts.preprocessed.map(lambda x: [ i if type(i)==str else i.decode("utf-8") for i in x])
                texts['embedding'] = texts.preprocessed.apply(model.infer_vector)
                for o, emb in texts[['objectId','embedding']].values:
                    f.write(str(o) + ',' + ','.join([str(xxx) for xxx in emb]) + "\n")

    test_texts = pd.read_parquet('../dataset/texts/test/' ,columns = ['objectId','preprocessed'], engine='fastparquet')        
    test_texts['preprocessed']=test_texts.preprocessed.map(lambda x: [ i if type(i)==str else i.decode("utf-8") for i in x])
    test_texts['embedding'] = test_texts.preprocessed.apply(model.infer_vector)
    for o, emb in test_texts[['objectId','embedding']].values:
        f.write(str(o) + ',' + ','.join([str(xxx) for xxx in emb]) + "\n")
#     parts.append(test_texts[['objectId','embedding']])

In [ ]:
d = all_[['objectId','embedding']].set_index('objectId').to_dict()['embedding']
np.save('doc2vecCache2', d)

In [ ]:
len(all_)

In [ ]:
all_['len'] = all_.preprocessed.map(lambda x: len(x))
d = all_[['objectId','len']].set_index('objectId').to_dict()['len']
np.save('tokenslenCache2', d)

In [ ]:
with open('doc2vec.csv', newline='') as f:
    content = f.readlines()
d = {}
for x in content:
    s= x.strip().split(",")
    d[s[0]]=[float(xxx) for xxx in s[1:]]


In [ ]:
np.save('doc2vec', d)

In [2]:
d = np.load('doc2vec.npy')

In [3]:
X =[]
K=[]
for k,x in d.item().items():
    X.append(x)
    K.append(k)
X=np.array(X)

In [ ]:
len(X[0])

In [4]:
from sklearn.cluster import KMeans,AgglomerativeClustering

In [ ]:
true_k = 200
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=30000, n_init=5)
model.fit(X)

In [ ]:
model2=AgglomerativeClustering(n_clusters=70)
model2.fit(X)

In [6]:
preds = model.predict(X)


In [19]:
[np.array(K),preds]

(3721891,)

In [20]:
np.array(K).shape

(3721891,)

In [7]:
pdClasses = pd.DataFrame(preds, columns=['text_cluster_120'], index = K)
pdClasses['instanceId_userId'] = pdClasses.index


In [8]:
pdClasses.to_hdf('text_cluster_120.h5', key='c', mode='w', format='table')

In [9]:
clusters = {}
for k,v in zip(K, preds):
    clusters[k]=v

In [10]:
np.save('clusters120', clusters)

In [11]:
c= np.load('clusters120.npy')

In [12]:

with open('clusters120.csv', 'a') as f:
    for k,v in c.item().items():
        f.write(str(k) + ','+  str(v) + "\n")


In [ ]:
test_texts = parquet.read_table('../texts/textsTest/', columns = ['objectId','preprocessed']).to_pandas()


In [ ]:
clusters['517288']

In [ ]:
cluster(28721520)

In [ ]:
cluster = {k for k,v in clusters.items() if  v == 5}
len(cluster)

In [ ]:
test_texts[(test_texts.objectId.map(lambda o : str(o) in cluster))]

In [ ]:
test_texts